In [2]:
import cv2
import os
import json
import glob
import shutil
from math import dist
import numpy as np

In [3]:
kpt_file = "../keypoint"
acupoints_name = ["HX6(肛門)", "HX3(尿道)", "HX1(耳中)", "TF4(神門)", "AH9(腰柢椎)", "AH11(胸椎)", "CO5(十二指腸)", "CO4(胃)", "CO3(賁門)", "CO14(肺)", "CO15(心)", "CO18(內分泌)", "HX2(輪2)", "AT2(緣中)", "LO8(扁桃體)", "LO1(牙)" , "LO2(舌)", "LO3(頷)", "LO4(垂前)", "LO5(眼)", "LO6(內耳)"]
ear_types = ["free","attached"]
tra_inp = "../traditional_inpainting"
degrees = ['15cm_0mm_0deg', '15cm_25mm_5deg', '15cm_50mm_10deg', '20cm_0mm_0deg', '20cm_25mm_5deg', '20cm_50mm_10deg']

In [4]:
for ear_type in ear_types:
    names = os.listdir(os.path.join(kpt_file, ear_type, "0_original_video"))
    for name in names:
        for deg in degrees:
            
            if not os.path.exists(os.path.join(tra_inp, ear_type, name, deg)):
                os.makedirs(os.path.join(tra_inp, ear_type, name, deg))
            count = 0
            original_video_name_deg = os.path.join(kpt_file, ear_type, "0_original_video", name, deg)
            video = os.path.join(original_video_name_deg, "resize.mp4")
            with open(os.path.join(kpt_file, ear_type, "2_json",name, deg, "mask.json"), "r") as f_mask:
                mask = json.load(f_mask)
            cap = cv2.VideoCapture(video)
            if not cap.isOpened():
                print("Cannot open camera")
                exit()

            while True:
                ret, frame = cap.read()
                if not ret:
                    print("Cannot receive frame")
                    break

                
#                 mask_ori_size = 255 - np.zeros((frame.shape[0], frame.shape[1]), np.uint8)
                mask_ori_size = np.zeros((frame.shape[0], frame.shape[1]), np.uint8)
                for i in range(0,len(mask["frame_0"])):
                    # imgRoi = frame[p1y:p2y,p1x:p2x]
                    imgRoi_mask = 255 - np.zeros((mask["frame_"+str(count)][str(i)]['right_bot'][1]- mask["frame_"+str(count)][str(i)]['left_top'][1], mask["frame_"+str(count)][str(i)]['right_bot'][0]- mask["frame_"+str(count)][str(i)]['left_top'][0]), np.uint8)
                    mask_ori_size[mask["frame_"+str(count)][str(i)]['left_top'][1] : mask["frame_"+str(count)][str(i)]['right_bot'][1], mask["frame_"+str(count)][str(i)]['left_top'][0] : mask["frame_"+str(count)][str(i)]['right_bot'][0]] = imgRoi_mask
                dst = cv2.inpaint(frame, mask_ori_size, 3,cv2.INPAINT_TELEA)



                cv2.imwrite(os.path.join(tra_inp, ear_type, name, deg, 'test{index}.png'.format(name_0 = name, index = str(count))), dst)




                count = count + 1


            cap.release()
            cv2.destroyAllWindows()
            


            

#             if os.path.exists(os.path.join(jsons_name_deg, "keypoint_location.json")):


#                 with open(os.path.join(jsons_name_deg, "visible.json"), "r") as f_visible:
#                     visible = json.load(f_visible)
#                 with open(os.path.join(jsons_name_deg, "keypoint_location.json"), "r") as f_keypoint_location:
#                     keypoint_location = json.load(f_keypoint_location)






#                 video = os.path.join(original_video_name_deg, "resize.mp4")
                

#             if 
#                 dst = cv2.inpaint（src，mask, inpaintRadius，flags）

Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receive frame
Cannot receiv